In [1]:
#使用TensorFlow 之前，首先导入它
import tensorflow as tf

In [2]:
import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
sess = tf.InteractiveSession()

In [4]:
# 为了高效地在Python 里进行数值计算，我们一般会使用像NumPy 这样用其他语言
# 编写的库件，在Python 外用其它执行效率高的语言完成这些高运算开销操作（如矩阵
# 运算）。但是，每一步操作依然会需要切换回Python 带来很大开销。特别的，这种开销
# 会在GPU 运算或是分布式集群运算这类高数据传输需求的运算形式上非常高昂。

# TensorFlow 将高运算量计算放在Python 外进行，同时更进一步设法避免上述的额
# 外运算开销。不同于在Python 中独立运行运算开销昂贵的操作，TensorFlow 让我们可
# 以独立于Python 以外以图的形式描述交互式操作。这与Theano、Torch 的做法很相似。

# 我们先从创建输入图像和输出类别的节点来创建计算图。
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])
# 这里的x和y并不代表具体值，他们是一个占位符(placeholder) — 当TensorFlow 运
# 行时需要赋值的变量。

# 虽然placeholder的shape参数是可选的，但有了它，TensorFlow 能够自动捕捉因数据
# 维度不一致导致的错误。

In [5]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [6]:
# Variable需要在session之前初始化，才能在session中使用。初始化需要初始值（本
# 例当中是全为零）传入并赋值给每一个Variable。这个操作可以一次性完成。

sess.run(tf.initialize_all_variables())

In [8]:
# 预测分类与损失函数
y = tf.nn.softmax(tf.matmul(x,W) + b)

In [9]:
# 在训练中最小化损失函数同样很简单。我们这里的损失函数用目标分类和模型预
# 测分类之间的交叉熵。
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

In [10]:
# 我们已经定义好了模型和训练的时候用的损失函数，接下来使用TensorFlow 来训
# 练。因为TensorFlow 知道整个计算图，它会用自动微分法来找到损失函数对于各个变
# 量的梯度。TensorFlow 有大量内置优化算法，这个例子中，我们用快速梯度下降法让交
# 叉熵下降，步长为0.01。
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [11]:
# train_step这个操作，用梯度下降来更新权值。因此，整个模型的训练可以通过反
# 复地运行train_step来完成。
for i in range(1000):
  batch = mnist.train.next_batch(50)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

# 每一步迭代，我们都会加载50 个训练样本，然后执行一次train_step，使用feed_dict
# ，用训练数据替换placeholder向量x和y_。注意，在计算图中，你可以用feed_dict来替代
# 任何张量，并不仅限于替换placeholder。

In [12]:
# 评估模型
# 首先，要先知道我们哪些label 是预测正确了。tf.argmax是一个非常有用的函数。它
# 会返回一个张量某个维度中的最大值的索引。例如，tf.argmax(y,1)表示我们模型对每
# 个输入的最大概率分类的分类值。而tf.argmax(y_,1)表示真实分类值。我们可以用tf.
# equal来判断我们的预测是否与真实分类一致。
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
# 这里返回一个布尔数组。为了计算我们分类的准确率，我们将布尔值转换为浮点数
# 来代表对、错，然后取平均值。例如：[True, False, True, True]变为[1,0,1,1]，计算出
# 平均值为0.75
accuracy = tf.reduce_mean(tf.cast(correct_prediction , "float"))

print accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels})

0.9092
